# 03 Rational Closure

This notebook defines a method to calculate if a statement (a twiddle statement) is entailed by the rational closure of a knowledge base.

In [1]:
import os
import sys


sys.path.append(os.path.join('..', 'common'))

from datatypes import KnowledgeBase, Literal, Atom, Top, Bot, Normally
from util import materialized, print_knowledge_base, entails
from ranked_models import statement_ranking, print_statement_ranking

In [2]:
f = Literal(Atom('f')) # flies
b = Literal(Atom('b')) # is a bird
r = Literal(Atom('r')) # is a robin
p = Literal(Atom('p')) # is a pengiun
w = Literal(Atom('w')) # has wings
bot = Bot()            # Falsum
top = Top()            # Verum

In [3]:
def minimal_ranked_entailment(knowledge_base: KnowledgeBase, statement: Normally):
    rank = statement_ranking(knowledge_base)
    r = len(rank) - (float('inf') in rank.keys())
    i = 0
    antecedent = statement.left
    while i < r:
        partial_knowledge_base = {statement.materialize() for j in range(i,r) for statement in rank[j]} | materialized(rank.get(float('inf'), set()))
        if not entails(partial_knowledge_base, -antecedent):
            break
        i += 1
    partial_knowledge_base = {statement.materialize() for j in range(i,r) for statement in rank[j]} | materialized(rank.get(float('inf'), set()))
    return entails(partial_knowledge_base, statement.materialize())

We've also prepared the examples from the slides:

In [4]:
K = {b / f, p / b}
print_knowledge_base(K)

{ b |~ f, p |~ b }


In [5]:
minimal_ranked_entailment(K, p / f)

True

In [6]:
K = {b / f, -(p >> b) / bot}
print_knowledge_base(K)

{ b |~ f, p → b }


In [7]:
minimal_ranked_entailment(K, p / f)

True

In [8]:
K = {b / f, -(p >> b) / bot, p / -f}
print_knowledge_base(K)

{ b |~ f, p |~ ¬f, p → b }


In [9]:
minimal_ranked_entailment(K, p/f)

False

In [10]:
K = {b / f, -(p >> b) / bot,-(r >> b) / bot, b/w, p / -f}
print_knowledge_base(K)

{ r → b, p |~ ¬f, p → b, b |~ f, b |~ w }


In [11]:
minimal_ranked_entailment(K, r/w)

True

In [12]:
minimal_ranked_entailment(K, p/w)

False

This is the more advanced bloodcells example:

In [13]:
m = Literal(Atom('m')) # mamalian red blood cells
v = Literal(Atom('v')) # vertebrate red blood cells
a = Literal(Atom('a')) # avian red blood cells
c = Literal(Atom('c')) # cell membrane
n = Literal(Atom('n')) # nucleus

In [14]:
K = {-(m >> v) / bot, -(a >> v) / bot, v / c, v / n, m / -n}
rank = statement_ranking(K)
print_knowledge_base(K)
print('-'*8)
print_statement_ranking(rank)

{ m |~ ¬n, v |~ c, m → v, v |~ n, a → v }
--------
0: { v |~ n, v |~ c }
1: { m |~ ¬n }
∞: { a → v, m → v }


In [15]:
minimal_ranked_entailment(K, m/c)

False